# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    def Search(graph, root): ##use DFS to search the components of the root
        rootlist=[]
        queue=[]
        previous=dict()
        def insearch(graph,root): 
            g = graph.neighbors(root) 
            rootlist.append(root) 
            for i in g:
                if not i in rootlist:
                    previous[i]=root
                    insearch(graph,i)
        insearch(graph,root) #call the insearch function
        return [previous,rootlist]
    
    def BFS(graph, root): ##use BFS to search the components of the root
        queue = []
        rootlist = []
        queue.append(root)
        rootlist.append(root)
        previous = dict()
        while len(queue) > 0:
            u = queue[0] ##put the element out of queue
            queue = queue[1:]
            for i in graph.neighbors(u):
                if i not in rootlist:
                    queue.append(i) ##put the elements into queue
                    rootlist.append(i)
                    previous[i] = u
        return [previous, rootlist]
    
    def Augmenting_Path(s, t, graph): ####find if there exists an augmenting path in a graph. If yes, return it 
        prev, rootlist = BFS(graph, s)
        if t in rootlist:
            path = [t]
            node = t
            while node != None:          
                if node in prev.keys():
                    path.append(prev[node])
                    node = prev[node]
                else: 
                    node = None
            path=path[::-1]
            return path
        else:
            return None
        
    def Residual(graph, flow): ## Given the graph and flow, construct a residual graph
        residual = nx.DiGraph() ##construct an empty directed graph
        for (v1, v2) in graph.edges():
            if flow[v1][v2] > 0:
                residual.add_edge(v2, v1)
                residual[v2][v1]['capacity'] = flow[v1][v2]
            if flow[v1][v2] < graph[v1][v2]['capacity']:
                residual.add_edge(v1, v2)
                residual[v1][v2]['capacity'] = graph[v1][v2]['capacity'] - flow[v1][v2]         
        return residual

    
    total = 0 ##calculate the total demands of all nodes and check if it is zero
    for node in graph.nodes():
        total = total + graph.node[node]['demand']
    if total != 0:
        raise nx.NetworkXUnfeasible("An error is thrown if there is no flow satisfying the demands.")
    
    ##construct a new graph T with a new source and a new sink
    T = graph.copy()
    T.add_node('S')
    nodes=T.nodes()
    for i in nodes:
        if i != 'S':
            if T.node[i]['demand'] < 0:
                T.add_edge('S', i)
                T.edge['S'][i]['capacity'] = -T.node[i]['demand']
    T.add_node('T')
    for i in nodes:
        if i != 'S' and i != 'T':
            if T.node[i]['demand'] > 0:
                T.add_edge(i, 'T')
                T.edge[i]['T']['capacity'] = T.node[i]['demand']
    
    ##Ford-Fulkerson
    keys = []
    values = []
    for v0 in T.nodes():
        initial_flow = dict() ##set the initial flow to each edge starting from v0
        for (v1, v2) in T.edges():
            if v0==v1:
                initial_flow[v2] = 0
        values.append(initial_flow)
        keys.append(v0)
    flow = dict()
    for i in range(0,len(keys)): ##add values to the flow
        v=keys[i]
        flow[v] = values[i]
    Tf = Residual(T, flow) ##construct the residual graph
    path = Augmenting_Path('S', 'T', Tf) #Find an augmenting flow

    while path != None:  ##computue a new flow if there exists a s-t path in Tf
        capacity = []
        l=len(path)
        for i in range(l-1): #### find the bottleneck capacity of the path
            v1 = path[i]
            v2 = path[i+1]
            capacity.append(Tf.edge[v1][v2]['capacity'])
        bottleneck = min(capacity)
        for i in range(l-1):
            v1 = path[i]
            v2 = path[i+1]
            if (v1, v2) in T.edges(): ##forward edge
                flow[v1][v2] = flow[v1][v2] + bottleneck
            else: ##back edge
                flow[v2][v1] = flow[v2][v1] - bottleneck       
        Tf_new = Residual(T, flow) ##construct a new residual graph
        path = Augmenting_Path('S', 'T', Tf_new)
        
    ##calculate the max flow and check if it equals to demand      
    maxflow = 0
    demand = 0
    edges=T.edges()
    nodes=T.nodes()
    for (v1, v2) in edges:
        if v1 == 'S':
            maxflow = maxflow + flow['S'][v2]
    for i in nodes:
        if i != 'S' and i != 'T':
            if T.node[i]['demand'] > 0:
                demand = demand + T.node[i]['demand']
    if demand != maxflow:
        raise nx.NetworkXUnfeasible("An error is thrown if there is no flow satisfying the demands.")
    else: ##delete source 'S' and sink 'T'
        keys=flow.keys()
        for i in keys:
            if 'T' in flow[i].keys():
                del flow[i]['T']
            if 'S' in flow[i].keys():
                del flow[i]['S']
        del flow['S']
        del flow['T']
        return flow
        



To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    f_in = {}
    f_out = {}
    f = {}
    keys=flow.keys()
    f_out = {i: 0 for i in keys} ##the flow-out dictionary
    f_in = {i: 0 for i in keys} ##the flow-in dictionary
    for v1 in keys:
        f_out[v1] = sum(flow[v1].values())
        for v2 in flow[v1].keys():
            f_in[v2] = f_in[v2] + flow[v1][v2]
    f = {i: 0 for i in keys}
    for i in flow.keys():
        f[i] = f_in[i] - f_out[i] ##A dict of the net flow into each node
    return f

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
